# Data Download

To download our data, we use Boto3 (AWS's Python API). Beyond that, we use pandas and numpy to manipulate data and use sqlite3/json to match up districts with responses. Finally we use sklearn for scaling and plotly for visualizations.

In [2]:
import boto3
import numpy as np
import pandas as pd
import sqlite3
import json

In [3]:
from sklearn.preprocessing import scale

In [4]:
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

#### Get Credentials

Credentials should be stored in a text file with the secret followed by the id on a different line.

In [5]:
key_id = None
key_secret = None
with open('credentials', "r") as cred_file:
    key_secret = cred_file.readline().strip()
    key_id = cred_file.readline().strip()

#### Initialization

In [6]:
session = boto3.Session(aws_access_key_id = key_id, aws_secret_access_key = key_secret, region_name='us-east-2')
dynamodb = session.resource('dynamodb')
table = dynamodb.Table('GerrymandrResponses')

#### Download/Parse

We then download all data from the table using scan(). In the future we might have to split up reads because as we get more users the table will obviously grow in size. Responses come through as a JSON.

In [7]:
items = table.scan()['Items']

In [8]:
items

[{'14': {'fair': True,
   'timeToDecide': Decimal('2.43115496635437'),
   'timestamp': '4/15/18, 1:31:21 PM Pacific Daylight Time',
   'viewedDemographics': False,
   'viewedEducation': False,
   'viewedGraph': False,
   'viewedIncome': False,
   'viewedMap': False,
   'viewedName': False,
   'viewedRace': False},
  '202': {'fair': True,
   'timeToDecide': Decimal('2.8891190290451'),
   'timestamp': '4/15/18, 1:31:19 PM Pacific Daylight Time',
   'viewedDemographics': False,
   'viewedEducation': False,
   'viewedGraph': False,
   'viewedIncome': False,
   'viewedMap': False,
   'viewedName': False,
   'viewedRace': False},
  '287': {'fair': True,
   'timeToDecide': Decimal('1.31926608085632'),
   'timestamp': '4/15/18, 1:31:23 PM Pacific Daylight Time',
   'viewedDemographics': False,
   'viewedEducation': False,
   'viewedGraph': False,
   'viewedIncome': False,
   'viewedMap': False,
   'viewedName': False,
   'viewedRace': False},
  'userID': 'us-east-2:f8d3b730-1f6a-4141-acb6-8c68

In [9]:
def pandify(a):
    """Technical term :) Makes the response dictionary into a pandas DataFrame"""
    
    ids = []
    fair = []
    tds = []
    ts = []
    v_demo = []
    v_ed = []
    v_gra = []
    v_inc = []
    v_map = []
    v_name = []
    v_race = []
    user = []
    
    # we keep track of the recorrds per user because we don't know how many times they have responded in
    # in advance
    count = 0
    for user_responses in a:
        for dist_id in user_responses:
            if dist_id != "userID":
                response = user_responses[dist_id]

                ids.append(int(dist_id))
                fair.append(response.get('fair', None))
                tds.append(float(response.get('timeToDecide', -1)))
                ts.append(response.get('timestamp', None))
                v_demo.append(response.get('viewedDemographics', None))
                v_ed.append(response.get('viewedEducation', None))
                v_gra.append(response.get('viewedGraph', None))
                v_inc.append(response.get('viewedIncome', None))
                v_map.append(response.get('viewedMap', None))
                v_name.append(response.get('viewedName', None))
                v_race.append(response.get('viewedRace', None))
                count += 1
        user += [user_responses['userID']] * count  # Python magic appends array
        count = 0
    
    # now we build a DataFrame
    frame = pd.DataFrame({'distID': ids, 'userID': user, 'fair': fair, 'TD': tds, 'TS': ts, 'vDemo': v_demo, 'vEdu': v_ed, 'vGraph': v_gra,
                         'vInc': v_inc, 'vMap': v_map, 'vName': v_name, 'vRace': v_race})
    return frame

In [10]:
frame = pandify(items)
frame.head()

,TD,TS,distID,fair,userID,vDemo,vEdu,vGraph,vInc,vMap,vName,vRace
0,1.319266,"4/15/18, 1:31:23 PM Pacific Daylight Time",287,True,us-east-2:f8d3b730-1f6a-4141-acb6-8c689e1d2aea,False,False,False,False,False,False,False
1,2.889119,"4/15/18, 1:31:19 PM Pacific Daylight Time",202,True,us-east-2:f8d3b730-1f6a-4141-acb6-8c689e1d2aea,False,False,False,False,False,False,False
2,2.431155,"4/15/18, 1:31:21 PM Pacific Daylight Time",14,True,us-east-2:f8d3b730-1f6a-4141-acb6-8c689e1d2aea,False,False,False,False,False,False,False
3,11.291522,"3/2/18, 10:20:08 AM Eastern Standard Time",67,True,us-east-2:54d728ce-95b0-4c31-b301-c95e22143586,True,False,True,True,False,None,False
4,26.709943,"3/1/18, 4:25:48 PM Eastern Standard Time",69,True,us-east-2:54d728ce-95b0-4c31-b301-c95e22143586,False,False,False,False,True,None,False


#### Aggregated by User

This is probably not useful but gives a good idea of what we can do.

In [11]:
def aggregate_by_user(df):
    sliced = df[['distID', 'fair', 'userID']] # we only need these columns
    num_users = df.userID.nunique() # get the number of users
    
    matrix = np.empty((num_users, 441), dtype=int) # define an empty matrix filled with nans
    matrix.fill(np.nan)
    index = {} # keep track of which row users are stored in
    
    for _, row in sliced.iterrows():
        if row.userID not in index:
            index[row.userID] = len(index)
            
        if row.fair:
            matrix[index[row.userID], int(row.distID)] = 1
        else:
            matrix[index[row.userID], int(row.distID)] = 0
    
    new_ind = [None] * num_users # reverse the index
    for user in index:
        new_ind[index[user]] = user
        
    frame = pd.DataFrame(matrix, index=new_ind)
    return frame

aggregate_by_user(frame).head()

,0,1,2,3,4,5,6,7,8,9,...,431,432,433,434,435,436,437,438,439,440
us-east-2:f8d3b730-1f6a-4141-acb6-8c689e1d2aea,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,...,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808
us-east-2:54d728ce-95b0-4c31-b301-c95e22143586,-9223372036854775808,-9223372036854775808,-9223372036854775808,0,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1,...,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808
us-east-2:79ba0bfa-b183-4433-8992-caaf4a7b0d17,-9223372036854775808,0,-9223372036854775808,-9223372036854775808,0,-9223372036854775808,1,-9223372036854775808,-9223372036854775808,-9223372036854775808,...,-9223372036854775808,-9223372036854775808,-9223372036854775808,1,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808


#### Load CSV

Next we load our compactness scores, which are provided as a CSV output of the Compactness notebook.

In [12]:
compactness = pd.read_csv('compactness.csv', index_col=0)
compactness.head()

,STATEFP,CD115FP,PPopper,CHull,LongVar,LatVar,BClark,XSym,YSym
0,2,00,-1.539333,21.040088,21.047353,19.828083,6.750436,-0.942557,-3.028228
1,10,00,1.972069,-0.057290,-0.048865,-0.112271,-1.151813,-0.249933,0.249403
2,30,00,1.385478,0.196834,-0.015619,1.308340,-0.801309,1.093344,1.552939
3,38,00,1.489508,0.056164,-0.037410,0.413603,-1.974687,2.013228,1.865749
4,46,00,2.016713,0.068255,-0.030917,0.820600,-1.139430,1.306477,1.979569


#### Load SQL/JSON

FIPS codes are also stored in a JSON, taken from the Scripts directory of the app repository.

In [13]:
fips_map = {}
with open('states.json') as js_file:
    fips_map = json.load(js_file)

We load the SQL database from the app to match up internal IDs with geography. In the future I plan to separate this out to another script/JSON file.

In [14]:
conn = sqlite3.connect('districts.sql')
curs = conn.cursor()

mapping = {}

for row in curs.execute('''SELECT id, number, state FROM districts;'''):
    i, number, state = row
    mapping[str(number)+state] = int(i) # numberState -> id

In [15]:
def get_id(row):
    cd115 = row.CD115FP
    if cd115 == 'ZZ':
        return None
    return mapping.get(str(int(cd115))+fips_map[str(row.STATEFP)])

In [16]:
compactness['distID'] = compactness.apply(get_id, axis=1)

In [17]:
nc = compactness[pd.notnull(compactness['distID'])] # remove id-less districts
nc.distID = nc.distID.astype('int')
nc = nc.set_index('distID')

In [18]:
nc.describe()

,STATEFP,PPopper,CHull,LongVar,LatVar,BClark,XSym,YSym
count,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000,437.000000
mean,27.723112,-0.006508,0.000767,0.000768,-0.004671,-0.005415,0.017098,0.019538
std,16.250723,0.984638,1.009112,1.009114,1.001898,1.001881,0.980826,0.977276
min,1.000000,-1.783652,-0.061518,-0.048931,-0.221122,-1.996085,-3.101399,-3.036078
25%,12.000000,-0.743379,-0.060573,-0.048867,-0.210982,-0.703214,-0.639026,-0.620420
50%,27.000000,-0.212788,-0.056588,-0.048583,-0.170906,-0.069447,0.143546,0.137772
75%,42.000000,0.613698,-0.046462,-0.047757,-0.055731,0.593119,0.764822,0.749830
max,72.000000,3.882557,21.040088,21.047353,19.828083,6.750436,2.142786,2.113769


In [19]:
nc = nc.sort_index()
nc.head()

,STATEFP,CD115FP,PPopper,CHull,LongVar,LatVar,BClark,XSym,YSym
distID,,,,,,,,,
0,6,40,-0.284821,-0.061426,-0.048927,-0.219651,0.055951,-0.765415,-0.861384
1,6,02,-0.563483,-0.026708,-0.048357,0.895210,0.473893,-0.318009,0.087526
2,6,31,-1.359767,-0.061039,-0.048838,-0.219508,1.451142,-0.385799,-0.823153
3,6,10,0.595813,-0.058149,-0.048714,-0.175429,0.126529,0.072523,-0.061987
4,6,09,0.749631,-0.059348,-0.048699,-0.193318,-0.512837,1.240484,0.693277


In [20]:
measures = ["PPopper", "CHull","LongVar", "LatVar", "BClark", "XSym", "YSym"]

for column in measures:
    nc[column] = scale(nc[column]) # scale measures

In [21]:
nc.describe()

,STATEFP,PPopper,CHull,LongVar,LatVar,BClark,XSym,YSym
count,437.000000,4.370000e+02,4.370000e+02,4.370000e+02,4.370000e+02,4.370000e+02,4.370000e+02,4.370000e+02
mean,27.723112,4.064890e-18,1.219467e-17,-4.064890e-18,4.064890e-18,1.219467e-17,2.032445e-17,2.845423e-17
std,16.250723,1.001146e+00,1.001146e+00,1.001146e+00,1.001146e+00,1.001146e+00,1.001146e+00,1.001146e+00
min,1.000000,-1.806939e+00,-6.179266e-02,-4.930655e-02,-2.162894e-01,-1.989210e+00,-3.183102e+00,-3.130251e+00
25%,12.000000,-7.492258e-01,-6.085501e-02,-4.924315e-02,-2.061568e-01,-6.972874e-01,-6.697167e-01,-6.555893e-01
50%,27.000000,-2.097389e-01,-5.690170e-02,-4.896102e-02,-1.661105e-01,-6.398526e-02,1.290678e-01,1.211223e-01
75%,42.000000,6.306042e-01,-4.685594e-02,-4.814150e-02,-5.102252e-02,5.980944e-01,7.632142e-01,7.481305e-01
max,72.000000,3.954267e+00,2.087323e+01,2.088040e+01,1.981787e+01,6.750896e+00,2.169726e+00,2.145384e+00


#### Aggregating Responses

In [22]:
summed = frame[["fair", "distID"]].groupby("distID").sum()
counts = frame[["fair", "distID"]].groupby("distID").count()

In [23]:
percents = summed.divide(counts)
percents.head()

,fair
distID,
1,0.0
3,0.0
4,0.0
6,1.0
9,1.0


In [24]:
combined = percents.join(nc, how="outer")

In [25]:
combined.head()

,fair,STATEFP,CD115FP,PPopper,CHull,LongVar,LatVar,BClark,XSym,YSym
distID,,,,,,,,,,
0,NaN,6,40,-0.282979,-0.061702,-0.049302,-0.214819,0.061321,-0.798724,-0.902439
1,0.0,6,02,-0.566314,-0.027258,-0.048737,0.899205,0.478956,-0.342049,0.069649
2,NaN,6,31,-1.375948,-0.061318,-0.049215,-0.214676,1.455489,-0.411244,-0.863274
3,0.0,6,10,0.612419,-0.058451,-0.049091,-0.170630,0.131847,0.056573,-0.083516
4,0.0,6,09,0.768815,-0.059640,-0.049077,-0.188506,-0.507050,1.248730,0.690196


In [26]:
fig = ff.create_scatterplotmatrix(combined[measures+["fair"]], height=800, width=800)
iplot(fig, filename='Combined Graph')